In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import netCDF4 as nc
import xarray as xr
import matplotlib.cm as cm
from scipy.interpolate import interp1d
from salishsea_tools import (nc_tools, gsw_calls,viz_tools)

/home/ssahu/anaconda3/lib/python3.5/site-packages/xarray/conventions.py:9: FutureWarning: The pandas.tslib module is deprecated and will be removed in a future version.
  from pandas.tslib import OutOfBoundsDatetime


In [2]:
west_bdy_2d = nc.Dataset('/ocean/ssahu/CANYONS/bdy_files/2d_west_twoless_m04.nc');

In [3]:
nc_tools.show_variables(west_bdy_2d)

odict_keys(['xbT', 'yb', 'time_counter', 'sossheig', 'vobtcrtx', 'vobtcrty'])


In [4]:
sossheig_full = west_bdy_2d.variables['sossheig'][:,:,:];
vobtcrtx_full = west_bdy_2d.variables['vobtcrtx'][:,:,:];
vobtcrty_full = west_bdy_2d.variables['vobtcrtx'][:,:,:];


print(vobtcrtx_full.shape, sossheig_full.shape)



Nt = sossheig_full.shape[0]
Ny = sossheig_full.shape[1]
Nx = sossheig_full.shape[2]

sossheig_new = np.reshape(a=sossheig_full, newshape= (Nt,1,Nx*Ny))
vobtcrtx_new = np.reshape(a=vobtcrtx_full, newshape= (Nt,1,Nx*Ny))
vobtcrty_new = np.reshape(a=vobtcrty_full, newshape= (Nt,1,Nx*Ny))

print(sossheig_new.shape)

(43, 4, 98) (43, 4, 98)
(43, 1, 392)


In [5]:
bdy_file = nc.Dataset('/ocean/ssahu/CANYONS/bdy_files/2d_west_flather_yBT_looped_twoless_m04.nc', 'w', zlib=True);

bdy_file.createDimension('xbT', sossheig_new.shape[2]);
bdy_file.createDimension('yb', 1);
bdy_file.createDimension('time_counter', None);

xbT = bdy_file.createVariable('xbT', 'int32', ('xbT',), zlib=True);
xbT.units = 'indices';
xbT.longname = 'indices along left boundary ordered from outwards to inside (increasing nbr)';

yb = bdy_file.createVariable('yb', 'int32', ('yb',), zlib=True);
yb.units = 'indices';
yb.longname = 'a strip of indices across all of left boundary';

time_counter = bdy_file.createVariable('time_counter', 'int32', ('time_counter',), zlib=True);
time_counter.units = 's';
time_counter.longname = 'time';

sossheig = bdy_file.createVariable('sossheig', 'float32', ('time_counter', 'yb', 'xbT'), zlib=True);
vobtcrtx = bdy_file.createVariable('vobtcrtx', 'float32', ('time_counter', 'yb', 'xbT'), zlib=True);
vobtcrty = bdy_file.createVariable('vobtcrty', 'float32', ('time_counter', 'yb', 'xbT'), zlib=True);


sossheig[...] = sossheig_new[...];
vobtcrtx[...] = vobtcrtx_new[...];
vobtcrty[...] = vobtcrty_new[...];

print(sossheig.shape)

bdy_file.close()

(43, 1, 392)


### The west and the South strips are alright but we need to flip the arrays for the north and the east beacuse the sliced sections need to be arranged according to the increasing order of nbr (outwards to inner in the bdy data files for nemo inputs)

In [6]:
north_bdy_2d = nc.Dataset('/ocean/ssahu/CANYONS/bdy_files/2d_north_twoless_m04.nc');

In [7]:
sossheig_full = north_bdy_2d.variables['sossheig'][:,:,:];
vobtcrtx_full = north_bdy_2d.variables['vobtcrtx'][:,:,:];
vobtcrty_full = north_bdy_2d.variables['vobtcrtx'][:,:,:];


print(vobtcrtx_full.shape, sossheig_full.shape)

Nt = sossheig_full.shape[0]
Ny = sossheig_full.shape[1]
Nx = sossheig_full.shape[2]

sossheig_flipped = np.flip(sossheig_full, axis=1)
vobtcrtx_flipped = np.flip(vobtcrtx_full, axis=1)
vobtcrty_flipped = np.flip(vobtcrty_full, axis=1)

sossheig_new = np.reshape(a=sossheig_flipped, newshape= (Nt,1,Nx*Ny))
vobtcrtx_new = np.reshape(a=vobtcrtx_flipped, newshape= (Nt,1,Nx*Ny))
vobtcrty_new = np.reshape(a=vobtcrty_flipped, newshape= (Nt,1,Nx*Ny))

print(sossheig_new.shape)

(43, 4, 68) (43, 4, 68)
(43, 1, 272)


In [8]:
bdy_file = nc.Dataset('/ocean/ssahu/CANYONS/bdy_files/2d_north_flather_yBT_looped_twoless_m04.nc', 'w', zlib=True);

bdy_file.createDimension('xbT', sossheig_new.shape[2]);
bdy_file.createDimension('yb', 1);
bdy_file.createDimension('time_counter', None);

xbT = bdy_file.createVariable('xbT', 'int32', ('xbT',), zlib=True);
xbT.units = 'indices';
xbT.longname = 'indices along north boundary ordered from outwards to inside (increasing nbr)';

yb = bdy_file.createVariable('yb', 'int32', ('yb',), zlib=True);
yb.units = 'indices';
yb.longname = 'a strip of indices across all of north boundary';

time_counter = bdy_file.createVariable('time_counter', 'int32', ('time_counter',), zlib=True);
time_counter.units = 's';
time_counter.longname = 'time';

sossheig = bdy_file.createVariable('sossheig', 'float32', ('time_counter', 'yb', 'xbT'), zlib=True);
vobtcrtx = bdy_file.createVariable('vobtcrtx', 'float32', ('time_counter', 'yb', 'xbT'), zlib=True);
vobtcrty = bdy_file.createVariable('vobtcrty', 'float32', ('time_counter', 'yb', 'xbT'), zlib=True);


sossheig[...] = sossheig_new[...];
vobtcrtx[...] = vobtcrtx_new[...];
vobtcrty[...] = vobtcrty_new[...];

print(sossheig.shape)

bdy_file.close()

(43, 1, 272)


In [9]:
right_bdy_2d = nc.Dataset('/ocean/ssahu/CANYONS/bdy_files/2d_right_twoless_m04.nc');

In [10]:
sossheig_full = right_bdy_2d.variables['sossheig'][:,:,:];
vobtcrtx_full = right_bdy_2d.variables['vobtcrtx'][:,:,:];
vobtcrty_full = right_bdy_2d.variables['vobtcrtx'][:,:,:];


print(vobtcrtx_full.shape, sossheig_full.shape)


Nt = sossheig_full.shape[0]
Ny = sossheig_full.shape[1]
Nx = sossheig_full.shape[2]

sossheig_flipped = np.flip(sossheig_full, axis=1)
vobtcrtx_flipped = np.flip(vobtcrtx_full, axis=1)
vobtcrty_flipped = np.flip(vobtcrty_full, axis=1)

sossheig_new = np.reshape(a=sossheig_flipped, newshape= (Nt,1,Nx*Ny))
vobtcrtx_new = np.reshape(a=vobtcrtx_flipped, newshape= (Nt,1,Nx*Ny))
vobtcrty_new = np.reshape(a=vobtcrty_flipped, newshape= (Nt,1,Nx*Ny))

print(sossheig_new.shape)



(43, 4, 98) (43, 4, 98)
(43, 1, 392)


In [11]:
bdy_file = nc.Dataset('/ocean/ssahu/CANYONS/bdy_files/2d_right_flather_yBT_looped_twoless_m04.nc', 'w', zlib=True);

bdy_file.createDimension('xbT', sossheig_new.shape[2]);
bdy_file.createDimension('yb', 1);
bdy_file.createDimension('time_counter', None);

xbT = bdy_file.createVariable('xbT', 'int32', ('xbT',), zlib=True);
xbT.units = 'indices';
xbT.longname = 'indices along right boundary ordered from outwards to inside (increasing nbr)';

yb = bdy_file.createVariable('yb', 'int32', ('yb',), zlib=True);
yb.units = 'indices';
yb.longname = 'a strip of indices across all of right boundary';

time_counter = bdy_file.createVariable('time_counter', 'int32', ('time_counter',), zlib=True);
time_counter.units = 's';
time_counter.longname = 'time';

sossheig = bdy_file.createVariable('sossheig', 'float32', ('time_counter', 'yb', 'xbT'), zlib=True);
vobtcrtx = bdy_file.createVariable('vobtcrtx', 'float32', ('time_counter', 'yb', 'xbT'), zlib=True);
vobtcrty = bdy_file.createVariable('vobtcrty', 'float32', ('time_counter', 'yb', 'xbT'), zlib=True);


sossheig[...] = sossheig_new[...];
vobtcrtx[...] = vobtcrtx_new[...];
vobtcrty[...] = vobtcrty_new[...];

print(sossheig.shape)

bdy_file.close()

(43, 1, 392)


In [12]:
south_bdy_2d = nc.Dataset('/ocean/ssahu/CANYONS/bdy_files/2d_south_twoless_m04.nc');

In [13]:
sossheig_full = south_bdy_2d.variables['sossheig'][:,:,:];
vobtcrtx_full = south_bdy_2d.variables['vobtcrtx'][:,:,:];
vobtcrty_full = south_bdy_2d.variables['vobtcrtx'][:,:,:];


print(vobtcrtx_full.shape, sossheig_full.shape)

Nt = sossheig_full.shape[0]
Ny = sossheig_full.shape[1]
Nx = sossheig_full.shape[2]

sossheig_new = np.reshape(a=sossheig_full, newshape= (Nt,1,Nx*Ny))
vobtcrtx_new = np.reshape(a=vobtcrtx_full, newshape= (Nt,1,Nx*Ny))
vobtcrty_new = np.reshape(a=vobtcrty_full, newshape= (Nt,1,Nx*Ny))

print(sossheig_new.shape)



(43, 4, 68) (43, 4, 68)
(43, 1, 272)


In [14]:
bdy_file = nc.Dataset('/ocean/ssahu/CANYONS/bdy_files/2d_south_flather_yBT_looped_twoless_m04.nc', 'w', zlib=True);

bdy_file.createDimension('xbT', sossheig_new.shape[2]);
bdy_file.createDimension('yb', 1);
bdy_file.createDimension('time_counter', None);

xbT = bdy_file.createVariable('xbT', 'int32', ('xbT',), zlib=True);
xbT.units = 'indices';
xbT.longname = 'indices along south boundary';

yb = bdy_file.createVariable('yb', 'int32', ('yb',), zlib=True);
yb.units = 'indices';
yb.longname = 'a strip of indices across all of south boundary';

time_counter = bdy_file.createVariable('time_counter', 'int32', ('time_counter',), zlib=True);
time_counter.units = 's';
time_counter.longname = 'time';

sossheig = bdy_file.createVariable('sossheig', 'float32', ('time_counter', 'yb', 'xbT'), zlib=True);
vobtcrtx = bdy_file.createVariable('vobtcrtx', 'float32', ('time_counter', 'yb', 'xbT'), zlib=True);
vobtcrty = bdy_file.createVariable('vobtcrty', 'float32', ('time_counter', 'yb', 'xbT'), zlib=True);


sossheig[...] = sossheig_new[...];
vobtcrtx[...] = vobtcrtx_new[...];
vobtcrty[...] = vobtcrty_new[...];

print(sossheig.shape)

bdy_file.close()

(43, 1, 272)
